In [ ]:
!pip3 install transformers

     |████████████████████████████████| 1.3MB 8.1MB/s 
     |████████████████████████████████| 2.9MB 51.6MB/s 
     |████████████████████████████████| 1.1MB 51.5MB/s 
     |████████████████████████████████| 890kB 52.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=98de34cfb49bc79a1dc7bf26d5a1dc8bf54fe6d9f160d50112e57f1f43fb0608
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from transformers import RobertaTokenizerFast, TFRobertaModel, RobertaConfig
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.utils import to_categorical# And pandas for data import + sklearn because you allways need sklearn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np

In [ ]:
model_name = 'roberta-base'
max_length = 80
config = RobertaConfig.from_pretrained(model_name, out_hidden_states)
config.output_hidden_states = True
tokenizer = RobertaTokenizerFast.from_pretrained(pretrained_model_name_or_path = model_name, config = config)# Load the Transformers BERT model
transformer_model = TFRobertaModel.from_pretrained(model_name, config = config)

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [ ]:
bert = transformer_model.layers[0]
input_ids = Input(shape=(max_length,), name='input_ids', dtype='int32')
inputs = {'input_ids': input_ids}
bert_model = bert(inputs)[1]
pooled_output = Dense(units=64, kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='out0')(bert_model)
outputs = Dense(units=1, kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='out1')(pooled_output)
model = Model(inputs=inputs, outputs=outputs, name='BERT')
model.summary()

Model: "BERT"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 80)]              0         
_________________________________________________________________
roberta (TFRobertaMainLayer) ((None, 80, 768), (None,  124645632 
_________________________________________________________________
out0 (Dense)                 (None, 64)                49216     
_________________________________________________________________
out1 (Dense)                 (None, 1)                 65        
Total params: 124,694,913
Trainable params: 124,694,913
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
data = open("/content/gdrive/My Drive/NLP_Proj/merged.txt")
fin = data.readlines()

In [ ]:
loss = BinaryCrossentropy(from_logits=False)
metric = BinaryAccuracy('accuracy')

optimizer = Adam(
    learning_rate=5e-05,
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)

model.compile(
    optimizer = optimizer,
    loss = loss,
    metrics = metric)


x = tokenizer(
    text=fin,
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True,
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = False,
    verbose = True)

out= []
for i in range(24661):
  out.append(1)
for i in range(63436-24661):
  out.append(0)

y=np.array(out)


In [ ]:
history = model.fit(
    x={'input_ids': x['input_ids']},
    y=y,
    validation_split=0.1,
    batch_size=64,
    epochs=1)


893/893 [==============================] - 985s 1s/step - loss: 0.5222 - accuracy: 0.8064 - val_loss: 0.3603 - val_accuracy: 0.8875


In [ ]:
df = pd.read_csv("/content/gdrive/My Drive/NLP_Proj/test.csv")

In [ ]:
list1 = list(df['col1'])
list2 = list(df['col2'])
flist = list1+list2
flist = set(flist)
flist = list(flist)

In [ ]:
flist[1]

'Optimization of the parameters of an MT system is now done using automatic metrics, primarily BLEU.'

In [ ]:
fin[0]

'The conversion uses head propagation rules to find the head on the right-hand side of the CFG rules, first proposed for English in (Magerman, 1995). $$$$$ Since most natural language rules are not absolute, the disambiguation criteria discovered in this work are never applied deterministically.\n'

In [ ]:

x_test = tokenizer(
    text=flist[1:],
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True,
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = False,
    verbose = True)
